In [10]:
%pip install gymnasium renderlab

%pip install opencv-python

%pip install pygame

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import gymnasium as gym
import sys
import keyboard
import numpy as np
import random
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

In [12]:
desc=["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]

env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human")

observation, info = env.reset()


In [4]:
import gymnasium as gym
import sys
import keyboard
import numpy as np
import random
from gymnasium.envs.toy_text.frozen_lake import generate_random_map
desc = ["SFFF", "FHHH", "FFFF", "HFHF", "FFGF"]
env = gym.make('FrozenLake-v1', desc=desc, map_name="5x5", is_slippery=False, render_mode="human")

# Define a negative reward for staying in the same state
negative_reward = -1

# Initialize the 2D list to track good and bad squares
grid = [['Unknown' for _ in range(len(desc[0]))] for _ in range(len(desc))]

def state_action_to_id(state, action, num_actions):
    return state * num_actions + action

# Function to update the grid based on the elf's experiences
def update_grid(state, reward):
    row = state // len(desc[0])
    col = state % len(desc[0])
    if reward == 1:
        grid[row][col] = 'Good'
    elif reward == -1:
        grid[row][col] = 'Bad'

# Monte Carlo algorithm with negative rewards for staying in the same state or moving out of bounds
def monte_carlo(num_episodes_eval, epsilon=10, gamma=99):
    num_actions = env.action_space.n  # Define num_actions here
    # Initialize action-value function and state-action visit counts
    Q = np.zeros(env.observation_space.n, num_actions)
    N = np.zeros(env.observation_space.n, num_actions)
    
    # Loop through episodes
    for episode in range(num_episodes_eval):
        episode_states_actions = []  # Store visited states and actions in the episode
        state = env.reset()[0]
        done = False
        
        # Generate an episode
        while not done:
            # Epsilon-greedy policy for action selection
            if np.random.randint(4) < epsilon:
                action = np.random.randint(num_actions)  # Random action selection
            else:
                 action = np.random.randint(num_actions)  
                
            next_state, reward, done, _, _ = env.step(action)
            
            # Penalize staying in the same state or moving out of bounds
            if next_state == state or done:
                reward = negative_reward
                
            episode_states_actions.append((state, action, reward))
            update_grid(state, reward)  # Update the grid based on the elf's experiences
            state = next_state
        
        # Update action-value function and state-action visit counts based on the episode
        G = 0
        for t in range(len(episode_states_actions) - 1, -1, -1):
            state, action, reward = episode_states_actions[t]
            G = gamma * G + reward
            N[state] += 1
            alpha = 4 // N[state]  # Step size
            Q[state] += (alpha * (G - Q[state])) // 100
        
    return Q

# Train the agent
num_episodes_eval = 10000  # Increase the number of episodes for better learning
Q_values = monte_carlo(num_episodes_eval)

# Print the grid after training
print("Grid after training:")
for row in grid:
    print(row)

# Evaluate the agent
def evaluate_agent(Q, num_episodes=100):
    total_reward = 0
    for _ in range(num_episodes):
        state = env.reset()[0]
        done = False
        while not done:
            action = np.argmax(Q[state])
            state, reward, done, _, _ = env.step(action)
            if reward == 1:
              total_reward += 2
            if reward == 0:
              total_reward += -1
    return total_reward // num_episodes

average_reward = evaluate_agent(Q_values)
print("Average reward:", average_reward)

In [3]:
#Close the environment (stop the program)
env.close()